# Pandora Ransomware
> Analysis of Pandora ransomware including some fun unpacking

- toc: true 
- badges: true
- categories: [pandora,ransomware,malware,unpacking,dumpulator,emulation]

> twitter: https://twitter.com/kienbigmummy/status/1504750051956240384/photo/1

## Overview 

Sample: `5b56c5d86347e164c6e571c86dbf5b1535eae6b979fede6ed66b01e79ea33b7b`

Sample is x64 and is on [malshare](https://malshare.com/sample.php?action=detail&hash=5b56c5d86347e164c6e571c86dbf5b1535eae6b979fede6ed66b01e79ea33b7b).

Unpacked sample [2619862c382d3e375f13f3859c6ab44db1a4bce905b4a617df2390fbf36902e7](https://malshare.com/sample.php?action=detail&hash=2619862c382d3e375f13f3859c6ab44db1a4bce905b4a617df2390fbf36902e7)

**References**
- f0wl blog [Pandora Ransomware](https://dissectingmalwa.re/blog/pandora/)

```
https://synthesis.to/2021/03/03/flattening_detection.html
https://blog.quarkslab.com/deobfuscation-recovering-an-ollvm-protected-program.html
https://github.com/eset/stadeo
https://eshard.com/posts/D810-a-journey-into-control-flow-unflattening
```

## Stage 1 Unpacking

Stage1 is just a modified UPX. We can unpack it with the following steps.
- removed X permissions from the first PE section memory
- run till exception
- excpetion EIP is OEP for PE 
- dump and reconstruct imports with Scylla

## Payload Obfuscation 

The ransomware has both obfuscated strings and control flow obfuscation. The obfuscated strings can be deobfuscated directly with an emulator call to the deobfuscation function. The cf obfuscation requires special attention.

### Our Approach to Control Flow Obfuscation 
It appears as though some sort of control flow flattening has been applied to the main function. The basic blocks (bb) that contain the actual payload code are accessed via a state machine. The dispatcher uses a state as a key which is used to calculate jumps between basic blocks. The dispatcher code uses a compare with the key/state to generate a conditional lookup in a hard coded jump table. The jump table contains an obfuscated address of the next bb. 

Our approach is to seperate the dispatcher bb from the payload bb. For each dispatcher bb the code is emulated with all conditions to generate the conditional jump addresses. The bb is then replaced with a simple compare and conditional jmp. Emulation is done with [Dumpulator](https://github.com/mrexodia/dumpulator).

** Once the dispatcher has been deobfuscated we should be able to see the control flow for the payload bb and futher simplify the dispatcher possibly removing it completely.

In [26]:
DUMP_FILE = '/tmp/pandora.dmp'

from dumpulator import Dumpulator
dp = Dumpulator(DUMP_FILE)

mapped base: 0x7ffe0000, size: 0x1000, protect: AllocationProtect.PAGE_READONLY
mapped base: 0x7ffe2000, size: 0x1000, protect: AllocationProtect.PAGE_READONLY
mapped base: 0xe3dcb35000, size: 0x7000, protect: AllocationProtect.PAGE_READWRITE
mapped base: 0xe3dccf8000, size: 0x3000, protect: None
mapped base: 0xe3dccfb000, size: 0x5000, protect: AllocationProtect.PAGE_READWRITE
mapped base: 0xe3dcdfb000, size: 0x3000, protect: None
mapped base: 0xe3dcdfe000, size: 0x2000, protect: AllocationProtect.PAGE_READWRITE
mapped base: 0xe3dcefb000, size: 0x3000, protect: None
mapped base: 0xe3dcefe000, size: 0x2000, protect: AllocationProtect.PAGE_READWRITE
mapped base: 0x22ad9340000, size: 0x10000, protect: AllocationProtect.PAGE_READWRITE
mapped base: 0x22ad9350000, size: 0x2000, protect: AllocationProtect.PAGE_READWRITE
mapped base: 0x22ad9360000, size: 0x1a000, protect: AllocationProtect.PAGE_READONLY
mapped base: 0x22ad9380000, size: 0x4000, protect: AllocationProtect.PAGE_READONLY
mapped 

```
r14 = FFFFFFFFAE6529F8

00007FF6C4B068E3   | 3D E1A53B17           | cmp eax,173BA5E1                                      | not part of cf
00007FF6C4B068E8   | BA D8000000           | mov edx,D8                                            | rdx = 0xd8
00007FF6C4B068ED   | BD 20010000           | mov ebp,120                                           | rbp = 0x120
00007FF6C4B068F2   | 48:0F4CD5             | cmovl rdx,rbp                                         | 
00007FF6C4B068F6   | 48:8B1411             | mov rdx,qword ptr ds:[rcx+rdx]  | table:00007FF6C4B69640 + rdx = rdx=00007FF7164B42E9
00007FF6C4B068FA   | 4C:01F2               | add rdx,r14                     |rdx=00007FF7164B42E9 + r14
00007FF6C4B068FD   | FFE2                  | jmp rdx                         | rdx = 00007FF6C4B06CE1
```

if the jump is relative it's for a code bb

if the jump is reg then it's a cf bb

if there is a ret this is the end

start = 0x00007FF6C4B067F0 
last_jmp = 0x00007FF6C4B0706C
end = 0x00007FF6C4B070D0  


build a table of bb the start is next head after jmp, end is jmp
emulate
patch the jmp for each table entry

**Produce the bb table**
This needs to be updated to both skip non-code, and to record if the bb is a `cmovl` or `cmovz`
```python
bb_table = []
ptr = 0x00007FF6C4B067F0
end = 0x00007FF6C4B070D0 
bb_start = ptr
while ptr <= end:
    if print_insn_mnem(ptr) == 'jmp': 
        op_type = idc.get_operand_type(ptr, 0)
        if op_type == o_reg:
            # This is a cf bb save it
            reg_name = print_operand(ptr, 0)
            bb_table.append((bb_start,ptr,reg_name))
            ptr = next_head(ptr)
            bb_start = ptr
        else:
            # This is code bb don't save it
            ptr = next_head(ptr)
            bb_start = ptr
    else:
        ptr = next_head(ptr)
    
        
        
    
```

**Modify bb with jmp addresses**

This needs to be updated to add conditional jmp and account for the compare and `cmovl` or `cmovz`
```python
import struct 

bb_jmp_table = [(140697838577648, 140697838577687, 'rdx', 140697838577840),
 (140697838577689, 140697838577710, 'rdx', 140697838578096),
 (140697838577712, 140697838577733, 'rdx', 140697838578329),
 (140697838577735, 140697838577761, 'rdx', 140697838578757),
 (140697838577763, 140697838577789, 'rcx', 140697838577648),
 (140697838577840, 140697838577861, 'rdx', 140697838577863),
 (140697838577863, 140697838577889, 'rdx', 140697838577891),
 (140697838577891, 140697838577917, 'rdx', 140697838578913),
 (140697838577919, 140697838577944, 'rcx', 140697838577648),
 (140697838578096, 140697838578122, 'rdx', 140697838578531),
 (140697838578124, 140697838578150, 'rdx', 140697838579331),
 (140697838578152, 140697838578178, 'rcx', 140697838577648),
 (140697838578224, 140697838578250, 'rdx', 140697838578252),
 (140697838578252, 140697838578278, 'rdx', 140697838578280),
 (140697838578280, 140697838578306, 'rcx', 140697838577648),
 (140697838578329, 140697838578355, 'rdx', 140697838579517),
 (140697838578357, 140697838578383, 'rcx', 140697838577648),
 (140697838578441, 140697838578462, 'rdx', 140697838578464),
 (140697838578464, 140697838578489, 'rcx', 140697838577648),
 (140697838578531, 140697838578557, 'rdx', 140697838579794),
 (140697838578559, 140697838578585, 'rcx', 140697838577648),
 (140697838578619, 140697838578645, 'rdx', 140697838578647),
 (140697838578647, 140697838578673, 'rcx', 140697838577648),
 (140697838578757, 140697838578783, 'rcx', 140697838577648),
 (140697838578913, 140697838578939, 'rcx', 140697838578941),
 (140697838579331, 140697838579357, 'rcx', 140697838577648),
 (140697838579420, 140697838579446, 'rcx', 140697838577648),
 (140697838579517, 140697838579542, 'rcx', 140697838577648),
 (140697838579720, 140697838579746, 'rcx', 140697838577648),
 (140697838579794, 140697838579820, 'rcx', 140697838577648)]
 
for bb in bb_jmp_table[1:]:

bb = (140697838577689, 140697838577710, 'rdx', 140697838578096)

ea_end = bb[1]
patch_start = prev_head(ea_end)
if ea_end - patch_start != 3:
    # We don't have enough space
    print(f"not enough space {hex(ea_end)}")
else:
    relative_jmp = bb[3] - (patch_start + 5)
    patch = b'\xe9' + struct.pack('<I',relative_jmp)
    patch_ptr = patch_start
    for c in patch:
        patch_byte(patch_ptr, c)
        patch_ptr += 1


```


In [25]:

# temp_addr = dp.allocate(256)
# dp.call(0x140001000, [temp_addr, 0x140017000])
# decrypted = dp.read_str(temp_addr)
# print(f"decrypted: '{decrypted}'")

# # Obfuscation key for table values
# dp.regs.r14 = 0xFFFFFFFFAE6529F8
# # Pointer to the jmp table 
# dp.regs.rcx = 0x00007FF6C4B69640
# dp.start(0x00007FF6C4B068E8,end=0x00007FF6C4B068FD)
dp = Dumpulator(DUMP_FILE)
bb_table = [(0x7ff6c4b067f0, 0x7ff6c4b06817, 'rdx'), (0x7ff6c4b06819, 0x7ff6c4b0682e, 'rdx'), (0x7ff6c4b06830, 0x7ff6c4b06845, 'rdx'), (0x7ff6c4b06847, 0x7ff6c4b06861, 'rdx'), (0x7ff6c4b06863, 0x7ff6c4b0687d, 'rcx'), (0x7ff6c4b068b0, 0x7ff6c4b068c5, 'rdx'), (0x7ff6c4b068c7, 0x7ff6c4b068e1, 'rdx'), (0x7ff6c4b068e3, 0x7ff6c4b068fd, 'rdx'), (0x7ff6c4b068ff, 0x7ff6c4b06918, 'rcx'), (0x7ff6c4b069b0, 0x7ff6c4b069ca, 'rdx'), (0x7ff6c4b069cc, 0x7ff6c4b069e6, 'rdx'), (0x7ff6c4b069e8, 0x7ff6c4b06a02, 'rcx'), (0x7ff6c4b06a30, 0x7ff6c4b06a4a, 'rdx'), (0x7ff6c4b06a4c, 0x7ff6c4b06a66, 'rdx'), (0x7ff6c4b06a68, 0x7ff6c4b06a82, 'rcx'), (0x7ff6c4b06a99, 0x7ff6c4b06ab3, 'rdx'), (0x7ff6c4b06ab5, 0x7ff6c4b06acf, 'rcx'), (0x7ff6c4b06b09, 0x7ff6c4b06b1e, 'rdx'), (0x7ff6c4b06b20, 0x7ff6c4b06b39, 'rcx'), (0x7ff6c4b06b63, 0x7ff6c4b06b7d, 'rdx'), (0x7ff6c4b06b7f, 0x7ff6c4b06b99, 'rcx'), (0x7ff6c4b06bbb, 0x7ff6c4b06bd5, 'rdx'), (0x7ff6c4b06bd7, 0x7ff6c4b06bf1, 'rcx'), (0x7ff6c4b06c45, 0x7ff6c4b06c5f, 'rcx'), (0x7ff6c4b06ce1, 0x7ff6c4b06cfb, 'rcx'), (0x7ff6c4b06e83, 0x7ff6c4b06e9d, 'rcx'), (0x7ff6c4b06edc, 0x7ff6c4b06ef6, 'rcx'), (0x7ff6c4b06f3d, 0x7ff6c4b06f56, 'rcx'), (0x7ff6c4b07008, 0x7ff6c4b07022, 'rcx'), (0x7ff6c4b07052, 0x7ff6c4b0706c, 'rcx')]
bb_jmp_table = []


## init the regs

dp.regs.r15 = 0x190
dp.regs.r14 = 0x0FFFFFFFFAE6529F8
dp.regs.r13 = 0x10
dp.regs.r12 = 0x1B0
dp.regs.rcx = 0x00007FF6C4B69640

## We need to calculate two jmp addresses conditional on the eax compare
## based on these we can replace the control bb with conditional jmps based on the compare
# def get_jmp(start_ea, end_ea, reg_name, jmp_cond):
#     dp.regs.r15 = 0x190
#     dp.regs.r14 = 0x0FFFFFFFFAE6529F8
#     dp.regs.r13 = 0x10
#     dp.regs.r12 = 0x1B0
#     dp.regs.rcx = 0x00007FF6C4B69640
#     if jmp_cond:
        
#     else:
        
#     dp.start(start_ea,end=end_ea)
#     print(hex(dp.regs.__getattr__(reg_name)))
    
    

for bb in bb_table:
    dp.regs.r15 = 0x190
    dp.regs.r14 = 0x0FFFFFFFFAE6529F8
    dp.regs.r13 = 0x10
    dp.regs.r12 = 0x1B0
    dp.regs.rcx = 0x00007FF6C4B69640
    print(f"BB {hex(bb[0])}:{hex(bb[1])}")
    dp.start(bb[0],end=bb[1])
    print(hex(dp.regs.__getattr__(bb[2])))
    bb_jmp_table.append((bb[0],bb[1],bb[2],dp.regs.__getattr__(bb[2])))

bb_jmp_table


mapped base: 0x7ffe0000, size: 0x1000, protect: AllocationProtect.PAGE_READONLY
mapped base: 0x7ffe2000, size: 0x1000, protect: AllocationProtect.PAGE_READONLY
mapped base: 0xe3dcb35000, size: 0x7000, protect: AllocationProtect.PAGE_READWRITE
mapped base: 0xe3dccf8000, size: 0x3000, protect: None
mapped base: 0xe3dccfb000, size: 0x5000, protect: AllocationProtect.PAGE_READWRITE
mapped base: 0xe3dcdfb000, size: 0x3000, protect: None
mapped base: 0xe3dcdfe000, size: 0x2000, protect: AllocationProtect.PAGE_READWRITE
mapped base: 0xe3dcefb000, size: 0x3000, protect: None
mapped base: 0xe3dcefe000, size: 0x2000, protect: AllocationProtect.PAGE_READWRITE
mapped base: 0x22ad9340000, size: 0x10000, protect: AllocationProtect.PAGE_READWRITE
mapped base: 0x22ad9350000, size: 0x2000, protect: AllocationProtect.PAGE_READWRITE
mapped base: 0x22ad9360000, size: 0x1a000, protect: AllocationProtect.PAGE_READONLY
mapped base: 0x22ad9380000, size: 0x4000, protect: AllocationProtect.PAGE_READONLY
mapped 

[(140697838577648, 140697838577687, 'rdx', 140697838577840),
 (140697838577689, 140697838577710, 'rdx', 140697838578096),
 (140697838577712, 140697838577733, 'rdx', 140697838578329),
 (140697838577735, 140697838577761, 'rdx', 140697838578757),
 (140697838577763, 140697838577789, 'rcx', 140697838577648),
 (140697838577840, 140697838577861, 'rdx', 140697838577863),
 (140697838577863, 140697838577889, 'rdx', 140697838577891),
 (140697838577891, 140697838577917, 'rdx', 140697838578913),
 (140697838577919, 140697838577944, 'rcx', 140697838577648),
 (140697838578096, 140697838578122, 'rdx', 140697838578531),
 (140697838578124, 140697838578150, 'rdx', 140697838579331),
 (140697838578152, 140697838578178, 'rcx', 140697838577648),
 (140697838578224, 140697838578250, 'rdx', 140697838578252),
 (140697838578252, 140697838578278, 'rdx', 140697838578280),
 (140697838578280, 140697838578306, 'rcx', 140697838577648),
 (140697838578329, 140697838578355, 'rdx', 140697838579517),
 (140697838578357, 14069